In [1]:
import numpy as np

from Bio.PDB import *
from Bio.PDB.PDBParser import PDBParser
import warnings
from Bio.PDB.PDBExceptions import PDBConstructionWarning
warnings.simplefilter('ignore', PDBConstructionWarning)

## Overview

This function takes in a PDB file and any contact map (in this case a DCA map) and prepares a tcl script for drawing lines between the contacts in VMD. Parameters are 'cutoff' and 'total_dca_pairs' which can be changed to show desired result.

- [Main function](#Generate-DCA-contacts-Main-Function)
- [Run function](#Run-function)

### Inputs

In [ ]:
pdbID = '2zwh'
pdb_file = '../actin/factin/oda9.pdb'
# pdb_file = '../actin/sbm/oda4mer/oda4.pdb'
# pdb_file = '../mreb/sbm/mamk/5ljv_mamk.pdb'
# pdb_file = '../hsp70/4jne.pdb'
# dca_contacts = np.loadtxt('../actin/pdb_interfacial_6A.contacts')
# dca_contacts = np.loadtxt('../actin/plmDCA/actinf150_mapped_plm.contacts')
dca_contacts = np.loadtxt('../actin/plmDCA/cytoplasm/from_actin-actin_subseq/cyt_oda_mapped.fn')
# dca_contacts = np.loadtxt('../mreb/plmdca/mreb_single/mamk_mapped.fn')
# dca_contacts = np.loadtxt('../hsp70/plmdca/hsp70_swiss_mapped.fn')

## Generate DCA contacts Main Function

[Back to Overview](#Overview)

In [ ]:
def generate_dca_interface(dca_contacts, cutoff, total_dca_pairs):
    # Initial parameters
    atomselect = 0
    valid_predictions = 0
    missing_res = 0
    
    # Extract PDB information using BIOpython
    p = PDBParser(PERMISSIVE=1)
    structure = p.get_structure(pdbID, pdb_file)
    
    chain_length = len(Selection.unfold_entities(structure, 'C'))
    
    target = open('./gen_dca_contacts.tcl', 'w')
    
    # Loop thru each DCA pair for each Chain in PDB and calculate pair distances
    # Then print out VMD instructions to draw contacts on PDB structure.
    for pair in dca_contacts[0:total_dca_pairs]:
        i = int(pair[0])
        j = int(pair[1])
#         print "i: %d, j: %d" % (i,j)
        chainsI = structure.get_chains()
        # Need this try statement in case there are missing residues in PDB
        try:    
            for chain_I in chainsI:
                chainsJ = structure.get_chains()
                for chain_J in chainsJ:
                    res_i = chain_I[i]
                    res_j = chain_J[j]
                    dist = abs(res_i['CA'] - res_j['CA'])
                    
                    if (chain_J != chain_I):
                        if (dist <= cutoff):
                            valid_predictions = valid_predictions + 1
#                             print "dist: %f\t Chain %s: %d, Chain %s: %d" % (dist, chain_I.id, i, chain_J.id, j)

                            target.write("set sel%d%s [atomselect top \"resid %d and name CA and chain %s\"]\n"
                                         % (i, chain_I.id, i, chain_I.id))
                            target.write("set sel%d%s [atomselect top \"resid %d and name CA and chain %s\"]\n" 
                                         % (j, chain_J.id, j, chain_J.id))

                            target.write("lassign [atomselect%d get {x y z}] pos1\n" % atomselect)
                            atomselect = atomselect + 1

                            target.write("lassign [atomselect%d get {x y z}] pos2\n" % atomselect)
                            atomselect = atomselect + 1

                            target.write("draw color magenta\n")
                            target.write("draw sphere $pos1 radius 2\n")
                            target.write("draw color pink\n")
                            target.write("draw sphere $pos2 radius 2\n")
                            target.write("draw color cyan\n")
                            target.write("draw line $pos1 $pos2 style dashed width 2\n")
                            
        # Need this in case there are missing residues in PDB
        except:
#             print "**NOTE: MISSING RESIDUES: %d AND %d" % (i, j)
            missing_res = missing_res + 1

    target.write("mol modselect 0 top \"all\"\n")
    target.write("mol modstyle 0 top newcartoon\n")
    target.write("mol modcolor 0 top chain\n")
    target.close()
    
    if (missing_res > 0):
        print "** NOTE: %d MISSING RESIDUES FROM PDB **\n" % missing_res
    print "Out of %d DCA predictions for %d chains:" % (total_dca_pairs, chain_length)
    print "Total DCA interfacial predictions at %dA cutoff: %d\n" % (cutoff, valid_predictions)

[Back to Overview](#Overview)

## Run function

In [ ]:
cutoff = 17
total_dca_pairs = 45

generate_dca_interface(dca_contacts, cutoff, total_dca_pairs)

[Back to Overview](#Overview)

[Back to Overview](#Overview)